# ERRA Climate Response Workshop (45 minutes)

This guided notebook introduces the Ensemble Rainfall-Runoff Analysis (ERRA) workflow, contrasts it with Budyko water-balance theory, and compares ERRA against a lightweight machine-learning benchmark.

## Session flow
- **0-5 min:** Concept recap (runoff generation, Budyko indices, ERRA intuition).
- **5-20 min:** Load regional case-study data (arid vs humid vs permafrost) and compute Budyko metrics.
- **20-30 min:** Fit ERRA kernels and interpret response differences.
- **30-38 min:** Stress-test scenarios (storms, droughts, reservoir control, consumptive use) and compare Budyko vs ERRA shifts.
- **38-45 min:** Machine-learning comparison (gradient-boosted trees) + wrap-up discussion.

## Data sources
The monthly hydroclimatic statistics in `data/regional_climate_responses.csv` are digitised from open archives: GRDC Tarim at Aksu (1967-2019 climatology), USGS Hudson River at Green Island (01463500 normals 1981-2010), and USGS Yukon River at Eagle (15356000 normals 1981-2010).

In [ ]:
from pathlib import Path
import pandas as pd

from code.examples import climate_response_case_studies as crcs
from code.examples import ml_vs_erra_comparison as ml_demo
from code import erra

In [ ]:
# Load regional case-study data
metrics = crcs.compute_budyko_metrics(crcs.load_case_study_data())
metrics.head()

## Budyko diagnostics
Compute evaporation index (E/P) and dryness index (PET/P), then compare against the Fu Budyko curve to highlight climatic contrasts.

In [ ]:
summaries = crcs.summarise_budyko_and_erra(
    metrics,
    {c: crcs.run_erra_for_catchment(metrics, c) for c in metrics['catchment'].unique()}
)
summary_df = pd.DataFrame([s.__dict__ for s in summaries])
summary_df

## Scenario exploration
Simulate extreme rainfall bursts, prolonged droughts, reservoir regulation, and consumptive water use to evaluate how Budyko indices and ERRA skill shift.

In [ ]:
scenario_outputs = crcs.evaluate_scenarios(crcs.generate_scenarios(metrics))
scenario_df = pd.DataFrame([s.__dict__ for s in scenario_outputs])
scenario_df.head(10)

## Machine-learning comparison
Train the handcrafted gradient-boosted stump ensemble (XGBoost-style) and compare cross-validated skill against ERRA forecasts.

In [ ]:
ml_comparisons = ml_demo.evaluate_ml_vs_erra(metrics)
ml_df = pd.DataFrame([c.__dict__ for c in ml_comparisons])
ml_df[['catchment', 'climate', 'erra_rmse', 'ml_rmse', 'erra_bias', 'ml_bias']]

## Discussion prompts
- Where does ERRA align with Budyko expectations, and where do they diverge?
- How do extreme events or management actions shift the water-balance indices?
- Which ML features dominate the boosted tree, and what hydrologic processes do they represent?
- How would you extend ERRA/ML with snowmelt or glacier inputs for high-elevation basins?